In [1]:
%run clean_data.ipynb

In [ ]:
possession_sequence = possession_df.assign(game_event_id=possession_df['game_event_id'].astype(int)).merge(all_games_event_keys, on='game_event_id', how='inner')

In [ ]:
def sequence_has_shot(sequence, event_id):
    
    # Get all events in the sequence
    events_in_sequence = possession_sequence[possession_sequence['sequence'] == sequence]
    
    # Get the index of the event
    event_index = events_in_sequence[
        events_in_sequence['game_event_id'] == event_id].index[0]
    
    # Check if any occurs after this index
    shot_after_event = events_in_sequence.query('index >= @event_index + 1 and possessionEventType == "SH"')
    
    if shot_after_event.empty:
        return 0
    return 1

In [ ]:
messi_touch_events = pd.to_numeric(events_df.query('gameId == 3835 and gameEventType == "OTB" and `player.nickname` == "Lionel Messi"')['game_event_id']).to_list()

messi_event_keys = all_games_event_keys.query('game_event_id in @messi_touch_events')

messi_sequences = messi_event_keys['sequence']

messi_event_pairs = list(zip(messi_sequences, messi_touch_events))

In [ ]:
shots_after_messi = pd.concat(
    [sequence_has_shot(sequence, event) for sequence, event in messi_event_pairs], ignore_index=True)